In [1]:
import re
import os
import pandas as pd
import requests
api_key = "###"

/Users/shelbyjouppi/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [15]:
df = pd.read_csv("MI-source-list-cleaned.csv")
df

,id,name,address,city,zip_code,county,full_address
0,N8277,TUSCOLA ENERGY INC - RUMBLE SHARP CTB,SE SW SW SEC 19 T14N R8E,AKRON,NaN,TUSCOLA,"SE SW SW SEC 19 T14N R8E, AKRON, MI"
1,N8274,TUSCOLA ENERGY INC-BOYCE JR TRUSTEE 1&STREETER 1,SW SW SW SEC 24 T14N R7E,AKRON,NaN,TUSCOLA,"SW SW SW SEC 24 T14N R7E, AKRON, MI"
2,U04060035,UNKNOWN,ALPENA COUNTY,ALPENA,NaN,ALPENA,"ALPENA COUNTY, ALPENA, MI"
3,N7824,RIVERSIDE - FOREST HOME 12,SW 1/4 NE 1/4 SEC 12 T30N R8W,BELLAIRE,NaN,ANTRIM,"SW 1/4 NE 1/4 SEC 12 T30N R8W, BELLAIRE, MI"
4,U15060046,UNKNOWN,CHARLEVOIX COUNTY,CHARLEVOIX,NaN,CHARLEVOIX,"CHARLEVOIX COUNTY, CHARLEVOIX, MI"
...,...,...,...,...,...,...,...
5832,B1966,WHITE PINE ELECTRIC POWER LLC,29639 WILLOW ROAD,WHITE PINE,49971.0,ONTONAGON,"29639 WILLOW ROAD, WHITE PINE, MI"
5833,P1069,ALTA EQUIPMENT COMPANY,1035 CALCITE ROAD,ROGERS CITY,49977.0,PRESQUE ISLE,"1035 CALCITE ROAD, ROGERS CITY, MI"
5834,P0067,LAURIE HAMMAR,15524 NORTH ROCK 38TH RD,ROCK,49980.0,DELTA,"15524 NORTH ROCK 38TH RD, ROCK, MI"
5835,N1973,"ABHE & SOVBODA, INC.",BLUE WATER BRIDGE & GRATIOT,PORT HURON,55372.0,SAINT CLAIR,"BLUE WATER BRIDGE & GRATIOT, PORT HURON, MI"


In [16]:
addresses = []
for cell in df['full_address']:
    addresses.append(cell)
addresses

['SE SW SW SEC 19 T14N R8E, AKRON, MI',
 'SW SW SW SEC 24 T14N R7E, AKRON, MI',
 'ALPENA COUNTY, ALPENA, MI',
 'SW 1/4 NE 1/4 SEC 12 T30N R8W, BELLAIRE, MI',
 'CHARLEVOIX COUNTY, CHARLEVOIX, MI',
 'SE 1/4 SE 1/4 SW 1/4 SEC 27, CHARLTON TWP, MI',
 'NE SW SE SEC 14 T30N R2W, CHESTER TWP, MI',
 'SW 1/4 SEC 31 T30N R2W, CHESTER TWP, MI',
 '18636 FITZPATRICK, DETROIT, MI',
 '11811 PLEASANT ST, DETROIT, MI',
 'NE 1/4 SEC 30 T31N R2W, DOVER TWP, MI',
 'SW 1/4 NW 1/4 SE 1/4 SEC 17, DOVER TWP, MI',
 '3010 W. MECHANIC ST., HILLSDALE, MI',
 '331 GRAHAM RD, IMLAY CITY, MI',
 'N. STATE ROAD, IONIA, MI',
 'ANY ROAD, IRON, MI',
 '8216 DOUGLAS AVE, KALAMAZOO, MI',
 '5944 E KILGORE RD, KALAMAZOO, MI',
 '34506 GLENDALE, LIVONIA, MI',
 '1708 E ASHMAN ST, MIDLAND, MI',
 'MONTMORENCY COUNTY, MONTMORENCY, MI',
 'MINISTRELLI CONSTRUCTION CO, NOVI, MI',
 'OTSEGO COUNTY, OTSEGO, MI',
 'SW 1/4 NE 1/4 SW 1/4 SEC 6, OTSEGO LAKE, MI',
 '5792 HOWELL RD., OTTER LAKE, MI',
 '5575 N RIVERVIEW, PARCHMENT, MI',
 '16350 

In [24]:
# Here is a loop that goes to different addresses
# And gathers up the information we want from the Google API requests
# time.sleep is putting in a one second pause between requests
# So that we don't overload Google's server (You want to do this if you're getting a lot of stuff)
# Because APIs often have limits on how many times you can send requests or how often

# this "list" (arrays are called lists in Python) gathers the results
results_array = []
import time

for place in addresses:
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json'
        keys = {'address': place, 'key': api_key}
        r = requests.get(url,params=keys)
        result_dic = r.json()
        print(place, result_dic['results'][0]['geometry']['location'])
        
        #better format for results, a dictionary (object) with address and geometry props
        
        each_result = {}
        each_result['address'] = place
        
        #make your geometry geojson friendly--make an array of [lng,lat] coordinates
        point_geo = []
        point_geo.append(result_dic['results'][0]['geometry']['location']['lng'])
        point_geo.append(result_dic['results'][0]['geometry']['location']['lat'])
        each_result['geometry'] = point_geo
        results_array.append(each_result)
        time.sleep(1)
    except:
        print("No Match")

SE SW SW SEC 19 T14N R8E, AKRON, MI {'lat': 43.5680758, 'lng': -83.5141259}
SW SW SW SEC 24 T14N R7E, AKRON, MI {'lat': 43.5678261, 'lng': -83.4647598}
ALPENA COUNTY, ALPENA, MI {'lat': 45.0616794, 'lng': -83.4327528}
SW 1/4 NE 1/4 SEC 12 T30N R8W, BELLAIRE, MI {'lat': 44.9802822, 'lng': -85.2111728}
CHARLEVOIX COUNTY, CHARLEVOIX, MI {'lat': 45.3180632, 'lng': -85.2584004}
SE 1/4 SE 1/4 SW 1/4 SEC 27, CHARLTON TWP, MI {'lat': 44.9953568, 'lng': -84.4340843}
NE SW SE SEC 14 T30N R2W, CHESTER TWP, MI {'lat': 43.158429, 'lng': -85.8690933}
SW 1/4 SEC 31 T30N R2W, CHESTER TWP, MI {'lat': 43.158429, 'lng': -85.8690933}
18636 FITZPATRICK, DETROIT, MI {'lat': 42.3688207, 'lng': -83.22270189999999}
11811 PLEASANT ST, DETROIT, MI {'lat': 42.279777, 'lng': -83.1415304}
NE 1/4 SEC 30 T31N R2W, DOVER TWP, MI {'lat': 41.87079440000001, 'lng': -84.16461629999999}
SW 1/4 NW 1/4 SE 1/4 SEC 17, DOVER TWP, MI {'lat': 44.1073663, 'lng': -85.62292959999999}
3010 W. MECHANIC ST., HILLSDALE, MI {'lat': 41.9

32000 32 MILE ROAD, LENOX, MI {'lat': 42.80316699999999, 'lng': -82.796776}
48175 GRAIOT AVENUE, CHESTERFIELD, MI {'lat': 42.6538797, 'lng': -82.8519651}
50925 RICHARD W BLVD, CHESTERFIELD, MI 4, CHESTERFIELD TOWNSHIP, MI {'lat': 42.6729774, 'lng': -82.8510208}
50271 E. RUSSELL SCHMIDT BLVD., NEW BALTIMORE, MI {'lat': 42.66424019999999, 'lng': -82.8508526}
29 WEST RUNDELL, PONTIAC, MI {'lat': 42.6518439, 'lng': -83.29591599999999}
930 COURT ST, PORT HURON, MI {'lat': 42.9714415, 'lng': -82.4315147}
4200 DOVE ROAD, PORT HURON, MI {'lat': 42.9527862, 'lng': -82.4907844}
1033 RIVER, PORT HURON, MI {'lat': 42.9848486, 'lng': -82.43630069999999}
1026 MILITARY, PORT HURON, MI {'lat': 42.9728532, 'lng': -82.4240239}
1631 DOVE ST., PT. HURON, MI {'lat': 42.952734, 'lng': -82.4426076}
68059 S. MAIN ST., RICHMOND, MI {'lat': 42.8022666, 'lng': -82.7497618}
43653 GRATIOT, CLINTON TWP, MI {'lat': 42.6163657, 'lng': -82.86929920000001}
129 W. LAFEYETTE, ROMEO, MI {'lat': 42.8013677, 'lng': -83.0147

40 S. MONROE, MONROE, MI {'lat': 41.9162229, 'lng': -83.3980716}
1011 S. MONROE, MONROE, MI {'lat': 41.9062787, 'lng': -83.4053935}
15488 EASTWOOD, MONROE, MI {'lat': 41.9030476, 'lng': -83.41707939999999}
7051 N. MONROE STREET, MONROE, MI {'lat': 41.9799275, 'lng': -83.373221}
7015 N. MONROE STREET, MONROE, MI {'lat': 41.9794889, 'lng': -83.37363839999999}
19600 WALTZ RD., HURON TOWNSHIP, MI {'lat': 42.1612069, 'lng': -83.40345459999999}
54250 GRAND RIVER AVENUE, NEW HUDSON, MI {'lat': 42.5062348, 'lng': -83.5864407}
53000 GRAND RIVER AVENUE, NEW HUDSON, MI {'lat': 42.50397359999999, 'lng': -83.5747087}
5475 GRAND RIVER AVENUE, NEW HUDSON, MI {'lat': 42.5773472, 'lng': -83.8440864}
30560 MILFORD ROAD, NEW HUDSON, MI {'lat': 42.5147418, 'lng': -83.61501249999999}
10200 TELEGRAPH, CARLETON, MI {'lat': 42.0248013, 'lng': -83.3415752}
81600 BELLE RIVER ROAD, MEMPHIS, MI {'lat': 42.9005916, 'lng': -82.7725928}
777 BASELINE ROAD, NORTHVILLE, MI {'lat': 42.43929720000001, 'lng': -83.4741073}

5792 HOLCOMB, DETROIT, MI {'lat': 42.3861328, 'lng': -83.0064617}
6415 IROQOUIS, DETROIT, MI {'lat': 42.3883441, 'lng': -83.0168036}
6432 IROQUOIS, DETROIT, MI {'lat': 42.388744, 'lng': -83.01636490000001}
6433 IROQUOIS, DETROIT, MI {'lat': 42.3885252, 'lng': -83.01692159999999}
6126 IROQUOIS, DETROIT, MI {'lat': 42.3871759, 'lng': -83.01521}
INTERSECTION OF TOWNSEND ST. AND I-94, DETROIT, MI {'lat': 42.4129181, 'lng': -82.94332419999999}
10010 GRATIOT, DETROIT, MI {'lat': 42.39421189999999, 'lng': -83.00421519999999}
10020 GRATIOT, DETROIT, MI {'lat': 42.394304, 'lng': -83.00403349999999}
6622-30 VAN DYKE AVE., DETROIT, MI {'lat': 42.388548, 'lng': -83.021277}
11719 CAMDEN, DETROIT, MI {'lat': 42.3997452, 'lng': -82.9881319}
14363 DACOSTA, CITY, MI {'lat': 42.3915581, 'lng': -83.2605109}
17534 MCDOUGALL, CITY, MI {'lat': 42.4229774, 'lng': -83.0707412}
25300 MICHIGAN AVENUE, DEARBORN, MI {'lat': 42.2981873, 'lng': -83.2847573}
624 CHARLOTTE, DETROIT, MI {'lat': 42.3425265, 'lng': -83.

1160 N. OPDYKE ROAD, AUBURN HILLS, MI {'lat': 42.656224, 'lng': -83.2443909}
281 COLLIER ROAD, AUBURN HILLS, MI {'lat': 42.6912272, 'lng': -83.2979218}
3300 ELIZABETH LAKE RD, WATERFORD, MI {'lat': 42.64050599999999, 'lng': -83.351288}
3965 CASS ELIZABETH RD, WATERFORD, MI {'lat': 42.6242375, 'lng': -83.36420799999999}
4290 WALTON ROAD, WATERFORD, MI {'lat': 42.6902767, 'lng': -83.3723862}
38555 HILLS TECH DRIVE, FARMINGTON HILLS, MI {'lat': 42.4896067, 'lng': -83.4278337}
23847 INDUSTRIAL PARK DRIVE, FARMINGTON HILLS, MI {'lat': 42.462739, 'lng': -83.4242471}
23409 INDUSTRIAL PARK COURT, FARMINGTON HILLS, MI {'lat': 42.4639798, 'lng': -83.41924230000001}
38700 GRAND RIVER AVENUE, FARMINGTON HILLS, MI {'lat': 42.4705139, 'lng': -83.4265826}
23400 HALSTED ROAD, FARMINGTON HILLS, MI {'lat': 42.4625411, 'lng': -83.4141536}
24505 INDOPLEX CIRCLE, FARMINGTON HILLS, MI {'lat': 42.4721851, 'lng': -83.3945459}
23850 FREEWAY PARK DRIVE, FARMINGTON HILLS, MI {'lat': 42.4628243, 'lng': -83.420410

...

9375 MT FREDERIC RD, GAYLORD, MI {'lat': 44.8951246, 'lng': -84.7865207}
8151 AMOCO RD, GAYLORD, MI {'lat': 44.9162267, 'lng': -84.6542766}
2443 LAKE MANUKA RD   SEC 6 OTSEGO LAKE TWP, GAYLORD, MI {'lat': 44.9487286, 'lng': -84.74655399999999}
4063 TROMBLEY RD, GAYLORD, MI {'lat': 44.9702056, 'lng': -84.6086249}
2998 RANGER LAKE RD, GAYLORD, MI {'lat': 44.987101, 'lng': -84.5874858}
600 EDELWEISS VILLAGE PARKWAY, GAYLORD, MI {'lat': 45.0226932, 'lng': -84.6953381}
EAST DIXON LAKE ROAD, GAYLORD, MI {'lat': 44.9888549, 'lng': -84.6255335}
540 S OTSEGO AVE, GAYLORD, MI {'lat': 45.0228543, 'lng': -84.6781544}
2646 DICKERSON RD, GAYLORD, MI {'lat': 44.9924374, 'lng': -84.68829819999999}
SE SW NE SEC 18 T31N R3W, GAYLORD, MI {'lat': 45.0275126, 'lng': -84.6747523}
NW NW NE SEC 13 T29N R03W, GAYLORD, MI {'lat': 41.4925374, 'lng': -99.9018131}
SEC 34 29N 04W, GAYLORD, MI {'lat': 45.0275126, 'lng': -84.6747523}
SW SW SW SEC 17 T32N R3W, GAYLORD, MI {'lat': 45.0275126, 'lng': -84.6747523}
10875 

NW SE NE SECTION 12 T30 R2W, CHESTER TWP, MI {'lat': 43.158429, 'lng': -85.8690933}
CHESTER 12 SKLARCZYK C4-13 T30N-R2W SEC 13 NE NE SE, CHESTER TWP, MI {'lat': 43.158429, 'lng': -85.8690933}
T30N-R2W, SECTION 32, CHESTER TWP, MI {'lat': 45.0011522, 'lng': -84.54793099999999}
SW SE SW SEC 18 T30N R02W, CHESTER TWP, MI {'lat': 43.158429, 'lng': -85.8690933}
NW NE SW SEC 4, CLEON TWP, MI {'lat': 44.4568465, 'lng': -85.8690933}
14490 BECKETT ROAD, JOHANNESBURG, MI {'lat': 45.014682, 'lng': -84.3823043}
6327  OLD STATE RD., JOHANNESBURG, MI {'lat': 44.91112, 'lng': -84.54196809999999}
NW 1/4 OF SW 1/4 OF NW 1/4 SECTION 12, T29N R2W, JOHANNESBURG, MI {'lat': 44.9857311, 'lng': -84.4556356}
11859 SPARR RD, JOHANNESBURG, MI {'lat': 45.0431186, 'lng': -84.4323198}
9320 BEAR LANE SAWYER RD, JOHANNESBURG, MI {'lat': 45.0532432, 'lng': -84.4908034}
6250  OLD STATE RD, JOHANNESBURG, MI {'lat': 44.91553400000001, 'lng': -84.5411307}
TURTLE LAKE RD, JOHANNESBURG, MI {'lat': 44.9377308, 'lng': -84.54

16794 S HUGGININ ROAD, KINROSS, MI {'lat': 46.260954, 'lng': -84.47450119999999}
KINCHELOE AFB FIRE TRAIN AREA, KINROSS, MI {'lat': 46.2664445, 'lng': -84.4663642}
NE4 SE4 SE4 FOREST TWP, TOWER, MI {'lat': 43.165979, 'lng': -83.528342}
9663 COOP ROAD, TOWER, MI {'lat': 45.36462299999999, 'lng': -84.29560599999999}
WHITEHOUSE TRAIL,SW 1/4 OF NW 1/4 SEC 12, DOVER TWP, MI {'lat': 44.3148443, 'lng': -85.60236429999999}
108 GARFIELD ST, VANDERBILT, MI {'lat': 45.1452956, 'lng': -84.66246579999999}
1444 ALEXANDER ROAD, VANDERBILT, MI {'lat': 45.171805, 'lng': -84.639241}
8250 MILL STREET, VANDERBILT, MI {'lat': 45.14754749999999, 'lng': -84.6642878}
8476 YUILL ROAD, VANDERBILT, MI {'lat': 45.1498494, 'lng': -84.6637045}
SW/4, NE/4, SEC 25 CORWITH TWP, VANDERBILT, MI {'lat': 45.1449453, 'lng': -84.66278199999999}
T32N, R2W, SECTION 28, VANDERBILT, MI {'lat': 45.14279030000001, 'lng': -84.6603091}
T30N-R2W, SECTION 28, CHESTER TWP, MI {'lat': 43.158429, 'lng': -85.8690933}
T29N-R2W, SECTION 15

1050 HIGHWAY 41 EAST, ISHPEMING, MI {'lat': 46.5016711, 'lng': -87.65189269999999}
445 GREENWOOD ST, ISHPEMING, MI {'lat': 46.4869055, 'lng': -87.68219049999999}
1 TILDEN MINE ROAD, ISHPEMING, MI {'lat': 46.4399147, 'lng': -87.6563583}
80 EAGLE MILLS ROAD, NEGAUNEE, MI {'lat': 46.5130831, 'lng': -87.5107056}
16017 SARYA ROAD, PELKIE, MI {'lat': 46.7936854, 'lng': -88.6166085}
244 SOUTH CEDAR STREET, MANISTIQUE, MI {'lat': 45.9553667, 'lng': -86.24682779999999}
335 CHIPPEWA AVE, MANISTIQUE, MI {'lat': 45.9581921, 'lng': -86.2571914}
5348 WEST US 2, MANISTIQUE, MI {'lat': 45.9687401, 'lng': -86.1577555}
115 RIVER STREET, MANISTIQUE, MI {'lat': 45.9586812, 'lng': -86.2472143}
415 S CHIPPEWA AVE, MANISTIQUE, MI {'lat': 45.9565352, 'lng': -86.25718189999999}
601 INTAKE PARK ROAD, MANISTIQUE, MI {'lat': 45.9733831, 'lng': -86.2471229}
5665 W HWY US2, MANISTIQUE, MI {'lat': 48.3548086, 'lng': -114.2386644}
620 DEER ST, MANISTIQUE, MI {'lat': 45.9634171, 'lng': -86.2600808}
103 RIVER STREET, M

1001 E LEAD STREET, BESSEMER, MI {'lat': 46.48031760000001, 'lng': -90.03822509999999}
23862 ONECO ROAD, CALUMET, MI {'lat': 47.1894201, 'lng': -88.49187180000001}
MINE ST, CALUMET, MI {'lat': 47.2656336, 'lng': -88.413327}
25830 DEPOT STREET, CALUMET, MI {'lat': 47.24056, 'lng': -88.450153}
ONE MILE NE OF CALUMET IN CALUMET TOWNSHIP, CALUMET, MI {'lat': 47.24659219999999, 'lng': -88.45400599999999}
51804 INDUSTRIAL DRIVE, CALUMET, MI {'lat': 47.1668223, 'lng': -88.5080668}
1800 TECUMSEH RD, CALUMET, MI {'lat': 47.2112154, 'lng': -88.4798272}
205 OSCEOLA ST, LAURIUM, MI {'lat': 47.2387168, 'lng': -88.4435429}
85 N BRADY AVE, CASPIAN, MI {'lat': 46.0660014, 'lng': -88.6271089}
23 SOUTH THIRD STREET, CRYSTAL FALLS, MI {'lat': 46.0959453, 'lng': -88.33082639999999}
151  OSS RD, CRYSTAL FALLS, MI {'lat': 46.0971304, 'lng': -88.4663833}
257 INDUSTRIAL PARK RD, CRYSTAL FALLS, MI {'lat': 46.2040955, 'lng': -88.4101568}
678 OLD M69, CRYSTAL FALLS, MI {'lat': 46.1046828, 'lng': -88.2840239}
134

In [26]:
df = pd.DataFrame(results_array)
df.to_csv("csv/MI-source-list-geocoded.csv", index=False)